# Introduction

Takes exploratory data and fits variations of the DDM.

DDM: average-signal

Details: Take every signal that the subject saw in a trial, and average their values. Feed that into a DDM as a time-invariant signal.

## Preamble

In [ ]:
# Install (package verification, PyDDM, timer, parallelization)
#!pip install paranoid-scientist
#!pip install pyddm
#!pip install pytictoc  
#!pip install pathos  

In [1]:
# Libraries
import os
from pytictoc import TicToc
import csv
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import pyddm as ddm
from pyddm import Model, Sample, FitResult, Fittable, Fitted, ICPoint, set_N_cpus
from pyddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayUniformMixture, LossRobustBIC
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

In [2]:
# Directories
datadir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-common-consequence\\data\\processed_data"
ddmdir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-common-consequence\\analysis\\outputs\\ddm"

In [3]:
# Parallel settings
ncpu = multiprocessing.cpu_count()-1 # always save one core
ncpu

11

## Import and Clean Raw Data

rawdata_in: odd trials used for in-sample data for model fitting.

rawdata_out: even trials used for out-sample data for model predictions.

In [16]:
# Import
data = pd.read_csv(datadir+"\\pilotdata.csv")

# Ceiling of Maximum RT for PyDDM
maxRT = math.ceil(max(data.rt))

# Display
data
maxRT

16

---
# DDM: Condition Invariant

## Fit the average stimulus DDM

In [9]:
# Create a drift subclass so drift can vary with stimulus.
class DriftRate(ddm.models.Drift):
  name = "Drift depends linearly on value difference"
  required_parameters = ["driftrate"] # Parameters we want to include in the model.
  required_conditions = ["vDiff_ab"] # The column in your sample data that modulates the parameters above.
  def get_drift(self, conditions, **kwargs):
    return self.driftrate * conditions["vDiff_ab"]

# Define the model.
model_ci = Model(name="Standard DDM that does not distinguish between AB and A'B' choices",
                 drift=DriftRate(driftrate=Fittable(minval=-1, maxval=1.5)),
                 noise=NoiseConstant(noise=Fittable(minval=.001, maxval=2)),
                 bound=BoundConstant(B=1),
                 IC=ICPoint(x0=Fittable(minval=-.99, maxval=.99)),
                 overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.1)),
                 dx=.01, dt=.01, T_dur=maxRT,  # dx: spatial grid for evidence space (-B to B, in dx bins), dt: time step in s. See Shin et al 2022 Fig 4 for why I set dx=dt.
                 choice_names=("A","B"))

In [10]:
# Interactive plot! Play with the variables!
vDiff_ab = np.sort(data.vDiff_ab.unique())
pyddm.plot.model_gui_jupyter(model=model_ci, conditions={"vDiff_ab":vDiff_ab.tolist()})

Output()

In [12]:
# Only run this if specified at the start. Otherwise, just load pre-saved weights.
print("DDM: Condition Invariant.")

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:

    # Progress tracker.
    print("=========================")
    print("Subject " + str(subnum))

    # Subset the data.
    subdata = data[data["subject"]==subnum]

    # Create a sample object from our data. Sample objects are the standard input for pyDDM fitting functions.
    ddm_data = Sample.from_pandas_dataframe(subdata, rt_column_name="rt", choice_column_name="choice_ab", choice_names=("A","B"))

    # Fit the model and show it off. Keep track of how long it took to estimate the parameters.
    clock = TicToc() # Timer
    clock.tic()
    set_N_cpus(ncpu) # Parallelize
    fit_model_ci = fit_adjust_model(sample=ddm_data, model=model_ci,
                                    fitting_method="differential_evolution",
                                    lossfunction=LossRobustBIC,
                                    verbose=False)
    clock.toc("Fitting subject " + str(subnum) + " took")
    display_model(fit_model_ci)

    # Save
    filename = ddmdir + "fit_model_ci_" + str(subnum) + ".txt"
    with open(filename, "w") as f:
      f.write(repr(fit_model_ci))

DDM: Condition Invariant.
Subject 9156374


KeyboardInterrupt: 

## Extract parameters and negative log likelihood for the model objects

In [ ]:
# Placeholders
model_ci_bic = []
model_ci_drift = []
model_ci_noise = []
model_ci_bias = []
model_ci_ndt = []

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:
    
    # Load
    filename = ddmdir + "fit_model_ci_" + str(subnum) + ".txt"
    with open(filename, "r") as f:
        model_loaded = eval(f.read())

    # Negative Log Likelihood.
    model_ci_bic.append(model_loaded.get_fit_result().value())
    
    # Fitted parameters.
    model_ci_drift.append(model_loaded.parameters()['drift']['driftrate'])
    model_ci_noise.append(model_loaded.parameters()['noise']['noise'])
    model_ci_bias.append(model_loaded.parameters()['IC']['x0'])
    model_ci_ndt.append(model_loaded.parameters()['overlay']['nondectime'])
    
d = {'bic':model_ci_bic, "drift":model_ci_drift, "noise":model_ci_noise, "bias":model_ci_bias, "ndt":model_ci_ndt}
indiv_model_ci = pd.DataFrame(data=d)
indiv_model_ci

## Means of BIC and Estimates

Confidence intervals assume normal distribution.

In [ ]:
summstats_model_ci = pd.DataFrame(data={"mean":indiv_model_ci.mean(), 
                                        "se":indiv_model_ci.sem(),
                                        "ci_lower":indiv_model_ci.mean()-1.96*indiv_model_ci.sem(),
                                        "ci_upper":indiv_model_ci.mean()+1.96*indiv_model_ci.sem()}).T
summstats_model_ci

---
# DDM: Separately by Block (p,r)

## Separate datasets by type (AB or ApBp)

In [15]:
data_ab = data[data.type=="AB"]
data_apbp = data[data.type=="ApBp"]

data_apbp

,subject,trial,type,rt,p,r,vDiff_ab,choice_ab,vA,vB,choice_lr,H
20,9156482,21,ApBp,6.9802,0.5,0.7,19.25,1,21,1.75,0,5
21,9156482,22,ApBp,6.0251,0.5,0.7,-7.00,1,21,28.00,0,80
22,9156482,23,ApBp,6.4902,0.5,0.7,5.25,1,21,15.75,0,45
23,9156482,24,ApBp,5.0237,0.5,0.7,-5.25,1,21,26.25,0,75
24,9156482,25,ApBp,4.6140,0.5,0.7,15.75,1,21,5.25,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...
1391,9158573,36,ApBp,1.2262,0.5,0.7,-1.50,0,9,10.50,0,50
1392,9158573,37,ApBp,1.2635,0.5,0.7,-12.00,0,9,21.00,1,100
1393,9158573,38,ApBp,0.9154,0.5,0.7,0.60,0,9,8.40,0,40
1394,9158573,39,ApBp,0.9328,0.5,0.7,-2.55,0,9,11.55,0,55


## Generate DDM Model

In [ ]:
# Create a drift subclass so drift can vary with stimulus.
class DriftRate(ddm.models.Drift):
  name = "Drift depends linearly on value difference"
  required_parameters = ["driftrate"] # Parameters we want to include in the model.
  required_conditions = ["vDiff_ab"] # The column in your sample data that modulates the parameters above.
  def get_drift(self, conditions, **kwargs):
    return self.driftrate * conditions["vDiff_ab"]

# Define the model.
model_ab = Model(name="Standard DDM fit to AB choices",
                 drift=DriftRate(driftrate=Fittable(minval=-1, maxval=1.5)),
                 noise=NoiseConstant(noise=Fittable(minval=.001, maxval=2)),
                 bound=BoundConstant(B=1),
                 IC=ICPoint(x0=Fittable(minval=-.99, maxval=.99)),
                 overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.1)),
                 dx=.01, dt=.01, T_dur=maxRT,  # dx: spatial grid for evidence space (-B to B, in dx bins), dt: time step in s. See Shin et al 2022 Fig 4 for why I set dx=dt.
                 choice_names=("A","B"))

model_apbp = Model(name="Standard DDM fit to ApBp choices",
                 drift=DriftRate(driftrate=Fittable(minval=-1, maxval=1.5)),
                 noise=NoiseConstant(noise=Fittable(minval=.001, maxval=2)),
                 bound=BoundConstant(B=1),
                 IC=ICPoint(x0=Fittable(minval=-.99, maxval=.99)),
                 overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.1)),
                 dx=.01, dt=.01, T_dur=maxRT,  # dx: spatial grid for evidence space (-B to B, in dx bins), dt: time step in s. See Shin et al 2022 Fig 4 for why I set dx=dt.
                 choice_names=("A","B"))